**Time Module**

In [ ]:
import time

print(time.time())

In [ ]:
print(time.perf_counter())

In [ ]:
secs = time.time()

print(time.gmtime(secs))

In [ ]:
print(time.localtime(secs))

In [ ]:
for _ in range(3):
	print('Tick')
	time.sleep(1)
	print('Tock')
	time.sleep(1)

**DateTime Module**

In [ ]:
import datetime

print(type(datetime.datetime.now()))

print(datetime.datetime.now())

dt = datetime.datetime(2015, 10, 21, 16, 29, 0)
print(dt.year, dt.month, dt.day)
print(dt.hour, dt.minute, dt.second)

In [ ]:
print(datetime.datetime.fromtimestamp(1000000))

print(datetime.datetime.fromtimestamp(time.time()))

In [ ]:
thanksgiving2022 = datetime.datetime(2022, 11, 24, 0, 0, 0)
newyears2023 = datetime.datetime(2023, 1, 1, 0, 0, 0)
nov24_2022 = datetime.datetime(2022, 11, 24, 0, 0, 0)

print(thanksgiving2022 == nov24_2022)
print(thanksgiving2022 > newyears2023)
print(newyears2023 > thanksgiving2022)
print(newyears2023 != nov24_2022)

In [ ]:
delta = datetime.timedelta(days=11, hours=10, minutes=9, seconds=8)

print(delta.days, delta.seconds, delta.microseconds)

print(delta.total_seconds())

print(str(delta))

In [ ]:
dt = datetime.datetime.now()
print(dt)

threeYears = datetime.timedelta(days=365*3)
print(dt + threeYears * 2)

In [ ]:
oct21st = datetime.datetime(2015, 10, 21, 16, 29, 0)

print(oct21st.strftime('%5y/%m/%d %H:%M:%S'))

print(oct21st.strftime('%I:%M %p'))

print(oct21st.strftime('%B of %y'))

In [ ]:
print(datetime.datetime.strptime('October 21, 2015', '%B %d, %Y'))

print(datetime.datetime.strptime('2015/10/21 16:29:00', '%Y/%m/%d %H:%M:%S'))

print(datetime.datetime.strptime("October of '15", "%B of '%y"))

print(datetime.datetime.strptime("November of '63", "%B of '%y"))

**Threading Module**

In [ ]:
#Run in w3schools
import threading
print('Start of program')

def takeANap():
  time.sleep(3)
  print('Wake up!')

threadObj = threading.Thread(target=takeANap)
threadObj.start()
print('End of program.')

In [ ]:
#Run in w3schools
def wakeUp(firstname, lastname, hours = 8):
  time.sleep(2)
  print('Wake up {} {}! You have slept for {} hours'.format(firstname,lastname,hours))

threadObj = threading.Thread(target=wakeUp, args=('Jim', 'Carrey'), kwargs={'hours': 10})

threadObj.start()
print('New thread started.')

In [ ]:
# Sync the threads
import threading
import time

print('Start of program.')
def someFunc():
  print('Doing something in an async thread')
  time.sleep(5)
  print('Async thread finished')

threadObj = threading.Thread(target=someFunc)
threadObj.start()

print('Doing something in the main thread')
time.sleep(2)
print('Main thread task finished. Wait for the async thread to finish.')

threadObj.join()
print('End of program.')


In [ ]:
# Moar threads!!
import threading
import time
print('Start of program.')

def someFunc(i):
  print(' Start doing something in thread {}'.format(i))
  time.sleep(3)
  print(' Thread {} finished'.format(i))

threads = [None] * 5
for i in range(len(threads)):
  print('Main thread creating thread {}'.format(i))
  # must pass in args as an iteratable, even there's only one arg
  threads[i] = threading.Thread(target=someFunc, args=(i,))
  threads[i].start()

# do some other stuff
print()
print('*'*50)

# try commenting out this for loop and see difference
for i in range(len(threads)):
  threads[i].join()
  print('Main thread waited for thread {} to end'.format(i))

print('End of program.')

In [ ]:
# Threads with a race condition
import threading
import time

print('Start of program.')
myList = []

def raceFunc(i):
  myList.append(i)
  # address of list: hex(id(myList)) - try to print it!
  print(' Start doing something in thread {}, '
  'with list {} (at {})'.format(i, myList, hex(id(myList))))
  time.sleep(3)
  # I want thread i to read back the same myList as line 12/13 but...
  print(' Thread {} finished with list {}'.format(i, myList))

threads = [None] * 5
for i in range(len(threads)):
  threads[i] = threading.Thread(target=raceFunc, args=(i,))
  threads[i].start()

# do some other stuff
print() # print an empty line
print('*' * 50)

[thread.join() for thread in threads]
print('End of program.')

**Queue Module**

In [ ]:
# queue module - simple consumer function
# using a synchronized queue class
# https://docs.python.org/3/library/queue.html
import queue
import threading
import time
import random
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(funcName)s - %(lineno)d - %(message)s')
# uncomment this line to suppress debug msg
#logging.disable(logging.DEBUG)

def consumer(id, q):
  """ Thread that will consume items in the queue"""
  while True:
    # Queue.get(block=True, timeout=None) remove and return an item from the queue
    # Default behavior: block if necessary until an item is available.
    item = q.get()

    logging.debug('consumer {} got {}, now: {}'.format(id, item, q.queue))
    if item is None:
      break

    # Do something with the item obtained
    time.sleep(random.randint(1, 20)/10)
    logging.debug("Item {} processed by consumer {}".format(item, id))
  logging.debug("Consumer {} shutting down".format(id))

startTime = time.perf_counter()
# Constructor for a FIFO queue in the queue module
# The queue module implements multi-producer, multi-consumer queues
# Queue object uses locks to temporarily block competing threads
q = queue.Queue()
threads = []
numThreads = 3

# Piece of code that acts as the producer
for msg in ['A', 'B', 'C', 'D', 'E', 'F']:
  # Queue.put(item, block=True, timeout=None) puts item into the queue
  # Default behavior: block if necessary until a free slot is available
  q.put(msg)
  logging.debug('after put: {}'.format(q.queue))

# Start consumers
for threadId in range(numThreads):
  threads.append(threading.Thread(target=consumer, args=(threadId, q)))
  threads[threadId].start()
  q.put(None) # Add one sentinel at the end of the queue for consumer thread
  logging.debug('started consumer {}, queue: {}'.format(threadId, q.queue))

[thread.join() for thread in threads]

endTime = time.perf_counter()
print("Program executed in {:.4f}".format(endTime-startTime))